# Example of parameter search

In [ ]:
import torch
import seaborn as sns

from time_deconv.time_deconv import *

## Parameter Configuration

In [ ]:
device = torch.device("cuda:0")
dtype = torch.float32
dtype_np = np.float32

## Train Base Model

In [ ]:
bulk_anndata_path = "/home/nbarkas/disk1/work/deconvolution_method/datasets/ebov/load_data_python/ebov_bulk.h5ad"
sc_anndata_path = "/home/nbarkas/disk1/work/deconvolution_method/datasets/ebov/load_data_python/ebov_sc.h5ad"

In [ ]:
with open(bulk_anndata_path, 'rb') as fh:
    bulk_anndata  = anndata.read_h5ad(fh)
with open(sc_anndata_path, 'rb') as fh:
    sc_anndata = anndata.read_h5ad(fh)

In [ ]:
# select samples only after or on tp 0
bulk_anndata = bulk_anndata[bulk_anndata.obs['dpi_time'] >= 0,]

In [ ]:
ebov_dataset = DeconvolutionDataset(
    sc_anndata = sc_anndata,
    sc_celltype_col = "Subclustering_reduced",
    bulk_anndata = bulk_anndata,
    bulk_time_col = "dpi_time",
    dtype_np = dtype_np,
    dtype = dtype,
    device=device,
    feature_selection_method = 'overdispersed_bulk_and_high_sc' #'overdispersed_bulk'
)

In [ ]:
pseudo_time_reg_deconv = TimeRegularizedDeconvolution(
    dataset=ebov_dataset,
    polynomial_degree = 20,
    basis_functions = "polynomial",
    device=device,
    dtype=dtype)

In [ ]:
type(pseudo_time_reg_deconv.dataset.bulk_raw_gex_mg)

In [ ]:
pseudo_time_reg_deconv.fit_model(n_iters=5_000, verbose=True, log_frequency=1000)

## Examine Outputs

In [ ]:
pseudo_time_reg_deconv.plot_loss()

In [ ]:
pseudo_time_reg_deconv.calculate_composition_trajectories(n_intervals = 1000)
pseudo_time_reg_deconv.plot_composition_trajectories()

In [ ]:
pseudo_time_reg_deconv.plot_phi_g_distribution()

In [ ]:
pseudo_time_reg_deconv.plot_beta_g_distribution()
matplotlib.pyplot.yscale('log')

# Synthetic Data

In [ ]:
import copy
from tqdm import tqdm

In [ ]:
def generate_evaluation_params(
    vary_param = ('deconvolution_params', 'polynomial_degree'),
    type = 'continuous',
    start_val = 1,
    end_val = 5,
    step = 1,
    n_reps = 10,
):
    default_params = get_default_evaluation_param(device, dtype, dtype_np)
    
    param_set = []
    
    if type == 'continuous':
        for val in range(start_val, end_val, step):
            for rep in range(n_reps):
                tmp_params = copy.deepcopy(default_params)
                tmp_params[vary_param[0]][vary_param[1]] = val
                param_set.append(tmp_params)
    
    return param_set

In [ ]:
evaluation_params = generate_evaluation_params()

In [ ]:
sweep_degree_results = evaluate_paramset(evaluation_params, sc_anndata, pseudo_time_reg_deconv, show_progress = True)

In [ ]:
# Plot the results 
polynomial_degree_list = []
l1_error_norm_list = []

for r in sweep_degree_results:
    polynomial_degree_list.append(r['param']['deconvolution_params']['polynomial_degree'])
    l1_error_norm_list.append(r['error']['L1_error_norm'].item())
    
res_df = pd.DataFrame({'polynomial_degree': polynomial_degree_list,
                      'l1_error_norm': l1_error_norm_list})

sns.boxplot(x=res_df['polynomial_degree'], y=res_df['l1_error_norm'])